In [1]:
import pydiscamb

# Load structure

Structures can be loaded using cctbx, from pdb-files or cif-files. Here, only pdb is shown, but loading cif is similar.
Additionally, Fobs can be read from a mtz-file, or from a cif. Only mtz is shown.

In [2]:
import iotbx.pdb
import mmtbx.model
from iotbx import reflection_file_reader

pdb_filename = "../../4znn.pdb"
pdb_inp = iotbx.pdb.input(file_name=pdb_filename)
model = mmtbx.model.manager(model_input=pdb_inp)
xrs = model.get_xray_structure()

# Scattering table is read from the structure
xrs.scattering_type_registry(table="electron")

# Load Fobs
mtz_filename = "../../4znn.mtz"
miller_arrays = reflection_file_reader.any_reflection_file(
    file_name=mtz_filename
).as_miller_arrays()
for ma in miller_arrays:
    if ma.info().label_string() == "FP,SIGFP":
        f_obs = ma

# Structure factor calculations

In [3]:
## The simplest usage: calculate structure factors in one call
fcalc = pydiscamb.calculate_structure_factors_IAM(xrs, d_min=2)

# TAAM is also available in this way:
fcalc = pydiscamb.calculate_structure_factors_TAAM(xrs, d_min=2)
fcalc

`fcalc` is a `scitbx.array_family.flex.complex_double`, and does not retain information about the corresponding hkls.
These can be re-constructed with cctbx:

In [4]:
# Get set of indices. This is the same way pydiscamb gets indices from d_min
s = xrs.structure_factors(d_min=2).miller_set()
fcalc = s.array(data=fcalc)
fcalc

crystal.symmetry(
    unit_cell=(17.93, 4.71, 33.03, 90, 94.33, 90),
    space_group_symbol="P 1 21 1"
  )
size: 486

# Advanced structure factor calculations

For simple calculations, a simple `d_min` can be sufficient. For more complex workflows, e.g. when you want to use specific indices, use `DiscambWrapper`:

In [5]:
wrapper = pydiscamb.DiscambWrapper(xrs)

# Either set d_min:
wrapper.set_d_min(2.0)
fcalc = wrapper.f_calc()

# Or, as a short-hand:
fcalc = wrapper.f_calc(2.0)

# Or set indices directly
wrapper.set_indices([(0, 1, 0), (2, 3, 1)])
fcalc = wrapper.f_calc()

# Also works with miller.indices()
wrapper.set_indices(f_obs.indices())
fcalc = wrapper.f_calc()
fcalc

Note that fcalc is still a flex array without knowledge of indices. 
The corresponding indices are available in `wrapper.hkl`, both when using `d_min` and `indices`.
As a short-hand, passing an array of structure factors directly to `f_calc` yields a miller array including indices:

In [6]:
wrapper.f_calc(f_obs)

crystal.symmetry(
    unit_cell=(17.93000031, 4.710000038, 33.02999878, 90, 94.33000183, 90),
    space_group_symbol="P 1 21 1"
  )
size: 1118

In [7]:
# DiscambWrapper.hkl is a read-only list of tuples of three ints
wrapper.hkl[:10]

[(-12, 0, 1),
 (-12, 0, 2),
 (-12, 1, 1),
 (-12, 1, 2),
 (-12, 1, 3),
 (-11, 0, 1),
 (-11, 0, 2),
 (-11, 0, 3),
 (-11, 0, 4),
 (-11, 0, 5)]

# TAAM
DiSCaMB's TAAM functionality is available:

In [8]:
taam_wrapper = pydiscamb.DiscambWrapper(xrs, method=pydiscamb.FCalcMethod.TAAM)
fcalc = taam_wrapper.f_calc(2.0)

More control for e.g. logging can be achieved by passing additional kwargs:

In [9]:
wrapper = pydiscamb.DiscambWrapper(
    xrs,
    # Below are all available parameters for the TAAM calculator.
    # The first three are set when passing `method=FCalcMethod.TAAM`,
    # but kwargs override these defaults. 
    # (Meaning we could say `method=FCalcMethod.IAM` and would still use TAAM, by setting `model="taam"`.)
    model="taam",
    electron_scattering=False,                                  # Whether to use Mott-Bethe to convert to electron scattering
    bank_path=pydiscamb.taam_parameters.get_default_databank(), # Path to TAAM parameters, this is the default value
    
    # Below parameters are additional, and are not set when specifying FCalcMethod.
    assignment_info="atom_type_assignment.log",     # Path to output log file for atom type assignment
    parameters_info="multipolar_parameters.log",    # Path to output log file for assigned multipolar parameters
    multipole_cif="structure.cif",                  # Path to output multipolar cif file
    unit_cell_charge=0,                             # Unit cell charge, default 0
    scale=True,                                     # Whether to scale the multipolar parameters based on unit cell charge, default True
)
fcalc = wrapper.f_calc(2.0)

In [10]:
# List available banks in the installation
banks = pydiscamb.get_TAAM_databanks()
banks

['/home/vife5188/Phenix/phenix-2.0rc1-5592/lib/python3.9/site-packages/pydiscamb/data/empty_TAAM_databank.txt',
 '/home/vife5188/Phenix/phenix-2.0rc1-5592/lib/python3.9/site-packages/pydiscamb/data/MATTS2021databank.txt']

# Structure factor gradients
For refinement purposes, DiSCaMB's gradient calculations are exposed

In [11]:
# Set up wrapper
wrapper = pydiscamb.DiscambWrapper(xrs)
wrapper.set_indices(f_obs.indices())

# Get derivatives of each Fcalc with respect to each site, adp, occupancy ect.
f_calc_derivatives = wrapper.d_f_calc_d_params()

# Output is list of derivative objects, indices correspond to indices in f_obs.indices()
single_f_calc_derivatives = f_calc_derivatives[0]
# Corresponding hkl is therefore f_obs.indices()[0]

# Result object contains lists of parameter derivatives.
# Indices correspond to scattereres in the structure
site_derivatives = single_f_calc_derivatives.site_derivatives
first_atom_xyz = site_derivatives[0]
# Corresponding scatterer ("first atom") is therefore xrs.scatterers()[0]

We can also calculate derivatives with respect to a target function, provided a `list` of the derivatives of the target with respect to each Fcalc

In [12]:
# First, get target
from mmtbx import f_model
model = f_model.manager(f_obs=f_obs, xray_structure=xrs)
target = model.target_functor()(compute_gradients=True)
d_target_d_fcalc = target.d_target_d_f_calc_work()

# Calculate gradients of target
gradients = wrapper.d_target_d_params(list(d_target_d_fcalc.data()))

# Output is now similar to the earlier output for a single hkl
# However, the indices now correspond directly to scatterers
single_atom_gradients = gradients[0]
# Corresponds to xrs.scatterers()[0]
xyz = single_atom_gradients.site_derivatives
adps = single_atom_gradients.adp_derivatives
# Can be 1 or 6 elements in the adp list, depending on xrs (uiso/uaniso)

For integration with cctbx, passing a `cctbx.miller.array` or `flex.complex_double` yields a packed `flex.double` of the derivatives specified in the scatterer flags

In [13]:
from cctbx import xray
xray.set_scatterer_grad_flags(
    scatterers=xrs.scatterers(),
    site=True,
)
wrapper = pydiscamb.DiscambWrapper(xrs)
gradients = wrapper.d_target_d_params(d_target_d_fcalc)
gradients